In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *
import importlib, pipeline_helper

In [ ]:
importlib.reload(pipeline_helper)
from pipeline_helper import allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
import pickle, gzip

### Initial Deployment

In [ ]:
name = 'sphere_cap_0.7'

In [ ]:
input_path = '../../data/{}.json'.format(name)

In [ ]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]

In [ ]:
curr_um.energy(UmbrellaEnergyType.Full)

In [ ]:
target_height_multiplier = input_data['target_spacing_factor']

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

In [ ]:
view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.update(scalarField = rod_colors)
view.show()

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
allEnergies(curr_um)

In [ ]:
allGradientNorms(curr_um)

In [ ]:
curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.LinearActuator

In [ ]:
import importlib, pipeline_helper
importlib.reload(pipeline_helper)

In [ ]:
from pipeline_helper import set_joint_vector_field, show_center_joint_normal, show_joint_normal

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
curr_um.targetDeploymentHeight = thickness * target_height_multiplier
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1e-1
curr_um.setHoldClosestPointsFixed(True)
curr_um.scaleInputPosWeights(1)

In [ ]:
curr_um.angleBoundEnforcement = umbrella_mesh.AngleBoundEnforcement.Penalty

In [ ]:
allGradientNorms(curr_um)

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    # if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
    dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
view.showScalarField(rod_colors)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-5
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-4
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-3
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-2
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-1
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e0
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e0
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
eqays.plot()

In [ ]:
allGradientNorms(curr_um)

### Initialize Design Optimization

In [ ]:
allEnergies(curr_um)

In [ ]:
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1e-3
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.1)

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = opt_opts, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
opt_opts.niter = 50

In [ ]:
results.success

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

In [ ]:
# optimizer.objective.terms[-1].term.activationThreshold = thickness * 1

In [ ]:
optimizer.beta = 5 * 1e6
optimizer.gamma = 1
optimizer.eta = 0
optimizer.zeta = 0# 1e1

In [ ]:
allDesignObjectives(optimizer)

In [ ]:
allDesignGradientNorms(optimizer)

In [ ]:
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

In [ ]:
original_design_parameters = rest_height_optimizer.params()

In [ ]:
import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

In [ ]:
from pipeline_helper import UmbrellaOptimizationCallback

In [ ]:
from umbrella_optimization import OptEnergyType

In [ ]:
from design_optimization_analysis import DesignOptimizationAnalysis
doptays = DesignOptimizationAnalysis(rest_height_optimizer)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
import time

In [ ]:
pipeline_helper.prev_time_stamp = time.time()

In [ ]:
uo = rest_height_optimizer.get_parent_opt()

In [ ]:
uo.equilibriumOptimizer.options.verbose = 1
#uo.equilibriumOptimizer.options.verboseWorkingSet = True
uo.equilibriumOptimizer.options.gradTol = 1e-10
# Hold the closest points fixed in the target-attraction term of the equilibrium solve:
# this seems to make the design optimization much more robust.
uo.setHoldClosestPointsFixed(True, False)

In [ ]:
tfview = pipeline_helper.TargetFittingVisualization(curr_um, uo.target_surface_fitter, view)
cb = pipeline_helper.UmbrellaOptimizationCallback(rest_height_optimizer, view, True, False, 1, rod_colors, doptays.record, tfview=tfview)

In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 10000, 0.005, 1e-5, cb, input_data["plate_edge_length"] / 30 * 32)

In [ ]:
rest_height_optimizer.beta, rest_height_optimizer.gamma, rest_height_optimizer.eta

In [ ]:
import time

In [ ]:
# pipeline_helper.prev_time_stamp = time.time()

In [ ]:
# solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 100, 1.0, 1e-2, cb, input_data["plate_edge_length"] / 30 * 32 * 0.8)

In [ ]:
# solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 100, 1.0, 1e-2, cb, -1)

In [ ]:
# solverStatus

In [ ]:
doptays.plot()

In [ ]:
j = curr_um.joint(0)

In [ ]:
j.jointType()

In [ ]:
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.T):
        if (curr_um.joint(i).alpha / np.pi > 0.5):
            print(curr_um.joint(i).alpha / np.pi)

In [ ]:
import pickle 
import gzip

pickle.dump(curr_um, gzip.open('../../output/{}_optimized_without_deployment_force_{}.pkl.gz'.format(name, time.strftime("%Y_%m_%d_%H_%M")), 'w'))
# load_um = pickle.load(gzip.open('test_pickle_um.pkl.gz', 'r'))

In [ ]:
max(rest_height_optimizer.params() * input_data['bbox_diagonal']), min(rest_height_optimizer.params() * input_data['bbox_diagonal'])

In [ ]:
input_data['bbox_diagonal']

In [ ]:
rest_height_optimizer.params() * input_data['bbox_diagonal']

In [ ]:
import time

In [ ]:
import load_jsondata
importlib.reload(load_jsondata)

In [ ]:
from load_jsondata import update_optimized_json
update_optimized_json(input_path, rest_height_optimizer.params(), output_json_path = '../../output/{}_optimized_params_{}.json'.format(name, time.strftime("%Y_%m_%d_%H_%M")), optim_spacing_factor = target_height_multiplier)

In [ ]:
rest_height_optimizer.defaultLengthBound()

In [ ]:
rest_height_optimizer.numParams()

In [ ]:
# rest_height_optimizer.params() - rest_height_optimizer.defaultLengthBound()

In [ ]:
# rest_height_optimizer.params()

### Undeployment

In [ ]:
use_pin = True

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-1
curr_um.uniformDeploymentEnergyWeight = 0
curr_um.targetDeploymentHeight = thickness * target_height_multiplier * 10
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 0
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(1)

In [ ]:
allEnergies(curr_um)

In [ ]:
OPTS.niter = 600

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
allEnergies(curr_um)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
allEnergies(curr_um)

In [ ]:
# for i in range(curr_um.numJoints()):
#     print(curr_um.joint(i).alpha)

In [ ]:
view.show()

In [ ]:
import pickle 
import gzip

pickle.dump(curr_um, gzip.open('../../output/{}_optimized_rest_state_equilibrium_{}.pkl.gz'.format(name, time.strftime("%Y_%m_%d_%H_%M")), 'w'))
# load_um = pickle.load(gzip.open('test_pickle_um.pkl.gz', 'r'))